In [ ]:
import numpy as np
import gwdistributions
from gwdistributions import distributions, event, generators, transforms
from gwdistributions.utils import Parameters


To Fix:
Errors:
 - How to set parameters for each distibution:
    If I try to use the parameters method I get an error saying Parameters object is not callable
 - When trying to create an event generator I get an error saying "variate=variate already produced!" if I try to pass the distributions into it

Stuff I'm unsure of:
 - Currently using a Lumonisity Distance power law, pretyy sure I want to use a redshift power law. There's transform classes to go between the two but not sure when to impliment it
 - I was previously using Sine distributions for inclination, but I haven't been able to find one in Reed's code
 - I was sampling in azimuth and zenith because bilby_pipe seemed to prefer that, Reed's code uses RA/Dec, not sure what the best way to remedy that is
 - Waveform default is IMRPhenomXPHM in Reed's code, not sure how to change it to IMRPhenomNSBH
 - Not sure if I am implimenting the SNR cutoff right

In [15]:
paramsm11 = {'min':2.5, 'max':17, 'mean':11.053, 'stdv':2.909}
paramsm12 = {'min':2.5, 'max':17, 'mean':13.687, 'stdv':0.570}
m11 = distributions.TruncatedGaussianMass1()
#m11.parameters(paramsm11)
m12 = distributions.TruncatedGaussianMass1()
#m12.parameters(paramsm12)
m1 = distributions.Mixture((m11, 0.895), (m12, 0.105))

m2 = distributions.Fixed()
#m2.parameters(1.7)

luminosity_distance = distributions.PowerLawLuminosityDistance()
paramslum = {'min':20, 'max':1000, 'pow':2.7}
#luminosity_distance.parameters(paramslum)

chi_11 = distributions.TruncatedGaussian()
paramschi11 = {'min':-1, 'max':1, 'mean':-0.5, 'stdv':0.08}
#chi_11.parameters(paramschi11)
chi_12 = distributions.TruncatedGaussian()
paramschi12 = {'min':-1, 'max':1, 'mean':0, 'stdv':0.01}
#chi_12.parameters(paramschi12)
chi_13 = distributions.TruncatedGaussian()
paramschi13 = {'min':-1, 'max':1, 'mean':0.5, 'stdv':0.08}
#chi_13.parameters(paramschi13)
chi_1 = distributions.Mixture((chi_12, 0.85), (chi_11, 0.075), (chi_13, 0.075))

chi_2 = distributions.Zero()
lambda_1 = distributions.Zero()
lambda_2 = distributions.Zero()
theta_jn = distributions.Uniform() # Should be Sine distribution
#theta_jn.parameters({'min':0, 'max':np.pi})
psi = distributions.Uniform()
#psi.parameters({'min':0, 'max':2*np.pi})
phase = distributions.Uniform()
#phase.parameters({'min':0, 'max':2*np.pi})
RAdec = distributions.location.IsotropicRADec()
geocent_time = distributions.FixedTime()
#geocent_time.parameters({1240215503.0})


dists = [m1, m2, luminosity_distance, chi_1, chi_2, lambda_1, lambda_2, theta_jn]
detect_prob = transforms.DetectionProbabilityGivenOptimalSNR(detectors=['L1', 'H1'],
                                                              threshold_snr = 10)
gen = generators.EventGenerator(dists, detect_prob)


AssertionError: variate=variate already produced!